In [ ]:
import   os
import   sys
sys.path.insert(0, os.path.abspath('.'))
from     ismrmrdUtils   import   rawMRutils
import   numpy          as       np
import   ismrmrd

## First - read in (fully sampled) EPI data from array coil

In [ ]:
dataHeader, dataArray, refDataArray = rawMRutils.returnHeaderAndData('./ScanArchive_EPI_converted.h5')

In [ ]:
dataArray.shape

In [ ]:
refDataArray.shape

In [ ]:
import matplotlib
import matplotlib.pyplot as mplt

In [ ]:

def computeN2GhostCorrection (angleData):

   x = np.linspace(-0.5,0.5,len(angleData))
   w = np.abs(angleData)
   A = np.zeros([x.size,3])

   A[:,0] = w 
   A[:,1] = w*x
   A[:,2] = w*x**2
   B      = w*np.angle(angleData) # magnitude-weighted phase data

   X,resid,rank,sigma = np.linalg.lstsq(A, B, rcond=None)

   oefit = X[0] + X[1]*x + X[2]*x**2

   return (np.exp(1j * oefit))



def loadNonComplexToComplex (nonComplexData):
   return (nonComplexData[0::2] + 1j * nonComplexData[1::2])



def oneDimTransform (data2Transform):
   return np.fft.fftshift(np.fft.fft(np.fft.fftshift(data2Transform)))


In [ ]:
nCoils   = refDataArray.shape[0]
nNavs    = refDataArray.shape[2]
nSlices  = refDataArray.shape[3]
nReps    = refDataArray.shape[4]

corrData = np.zeros(refDataArray.shape, dtype=np.complex64)

for s in range (nSlices):
   for r in range (nReps):
      for n in range (1, nNavs, 1): # First line (n == 0) does NOT contain reference data, so can use to store results.
         corrData[:, :, n, s, r] = np.fft.fftshift(np.fft.fft(np.fft.fftshift(refDataArray[:, :, n, s, r], axes=[1]), axis=1), axes=[1])

      # Store odd - even phase difference back in line '0' of reference data array, as that line does not hold data useful.
      for c in range (nCoils):
         angleData = (corrData[c, :, 1, s, r] + corrData[c, :, 3, s, r]) * 0.5 * np.conj(corrData[c, :, 2, s, r])

         # Use phase difference between odd/even lines directly, normalized by magnitude (suggested by @jad11).
         # corrData[c, :, 0, s, r] = angleData / abs(angleData)

         # Or call polynomial phase fitting routine.
         corrData[c, :, 0, s, r] = computeN2GhostCorrection(angleData)

In [ ]:
# Debugging cell - checking contents of reference lines and results of computations to fit phase.

fig, ax     = mplt.subplots(1, 1, sharex=True, sharey=True)
# item2Plot   = np.zeros ((dataArray.shape[1], 2) , dtype = np.complex64)
item2Plot   = np.zeros ((100, 4) , dtype = np.complex64)

item2Plot = corrData[15, 70:170, 0, 1, 3]
# item2Plot = np.exp(-1j * oefit)

ax.plot((np.angle(item2Plot)), label=str('phase'))
# ax.plot(np.abs(item2Plot), label=str('mag'))
ax.legend()

In [ ]:
# Apply phase corrections.

correctedArray   = np.zeros (dataArray.shape, dtype = np.complex64)

xFormedInReadout = np.fft.fftshift(np.fft.fft(np.fft.fftshift(dataArray, axes=[1]), axis=1), axes=[1])

# Apply phase correction to alternating lines.
correctedArray[:, :, 0:dataArray.shape[2]:2, :, :] = (xFormedInReadout[:, :, 0:dataArray.shape[2]:2, :, :]
                                                          *   corrData[:, :, 0:1:1,                  :, :])

# Then copy in data that were not corrected, i.e. alternate lines NOT corrected above.
correctedArray[:, :, 1:dataArray.shape[2]:2, :, :] =  xFormedInReadout[:, :, 1:dataArray.shape[2]:2, :, :]

In [ ]:
imageSpace = (np.fft.fft(np.fft.fftshift(correctedArray, axes=[2]), axis=2))
# imageSpace = np.fft.fftshift(np.fft.fft2(dataArray, axes=(1,2)), axes=(1,2))
rawMRutils.computeAndPlot(imageSpace)
plottedFigures = mplt.figure(figsize=(12,18))

subImages = plottedFigures.add_subplot(1, 1, (1))

# fixedKSpace = np.fft.ifftshift(np.fft.ifft2(imageSpace, axes=(1,2)), axes=(1,2))
# disp = np.angle(fixedKSpace[15, :, :, 1, 3])
disp = abs(imageSpace[:, :, :, 1, 3])

subImages.imshow(np.sqrt(np.sum(disp*disp, axis=0)), 'viridis')
# subImages.imshow(disp)